## Oprava výpočtu nehodovosti

CBA nezohľadňuje
- neohlásené dopravné nehody
- počet zranených v nehode

Vo výpočte nehodovosti je ďalej použité nevhodné priemerovanie.

Pre zachovanie štruktúry hárkov sa oprava deje na dvoch miestach
- hárok 09-A Nehodovost - vypocet - korekcia počtu nehôd a oprava výpočtu celkovej nehodovosti
- hárok 09 Nehodovost - v záujme zachovania tabuliek Počet nehôd a Náklady na nehodovosť je počet zranených zohľadnený v prepočte medzi tabuľkami

In [ ]:
import numpy as np
import pandas as pd
import openpyxl as opx
from os import listdir


In [ ]:
# cesta k CBA excelom
base_path = 
# cesta k vystupu
out_path = 

# pozri dir
dir_contents = listdir(base_path)
# nechaj excely
xlsx_files = list(filter(lambda x: 'xlsx' in x, dir_contents))

In [ ]:
for cba_filename in xlsx_files:
    # otvor excel
    wb = opx.load_workbook(base_path + cba_filename)

    # Oprava frekvencie nehôd
    # CBA nezohľadňuje nenahlásené dopravné nehody.
    # Tabuľka nekorigovaných hodnôt je na hárku `09-A Nehodovost - vypocet` v bunkách `N552:Q563`.
    # Do tabuľky vpíšeme korigované hodnoty vypočítané z hodnôt v metodike uložených na hárku `A1 Parametre`.

    sheet_name = '09-A Nehodovost - vypocet'
    ws = wb[sheet_name]

    # top left
    tl_col = 'N'
    tl_row = '552'
    # bottom right
    br_col = 'Q'
    br_row = '563'

    tab_nehodovost = ws[tl_col+tl_row:br_col+br_row]
    n_rows = len(tab_nehodovost)
    n_cols = len(tab_nehodovost[0])

    # stlpce v tabulke frekvencie nehod na useku cesty
    cols_lett = 'CDEF'

    # iteruj stlpce - typ nehody
    for id_col in range(n_cols):
        # zakladna bunka je v prvom riadku
        base_cell_coord = tab_nehodovost[0][id_col].coordinate

        col_lett_nehoda = cols_lett[id_col]

        # vzorec
        # zaznamenana frekvencia nehod * korekcny faktor
        base_formula = '=Parametre!' + col_lett_nehoda + '149' + '*' + \
                'Parametre!$C$' + str(140+id_col) # korekcia pre nehodu v riadkoch
        # iteruj riadky - typ cesty
        for row in range(0, n_rows):
            current_coord = tab_nehodovost[row][id_col].coordinate
            tab_nehodovost[row][id_col].value = opx.formula.translate.Translator(base_formula, \
                                            origin=base_cell_coord).translate_formula(current_coord)


    # Oprava výpočtu počtu nehôd

    # CBA počíta priemernú nehodovosť (hustotu nehôd) na jednotkovú dĺžku. 
    # Výpočet nie je správny, keďže jednotkové hodnoty nehodovosti sú udávané na vozkm. 

    # Je potrebné výpočet opraviť.     

    # Počet nehôd vo variante V0

    # Počty nehôd podľa závažnosti v jednotlivých rokoch sú v bunkách `K422:BC425`.

    sheet_name = '09-A Nehodovost - vypocet'
    ws = wb[sheet_name]

    # top left
    tl_col = 'K'
    tl_row = '422'
    # bottom right
    br_col = 'BC'
    br_row = '425'

    tab_nehodovost = ws[tl_col+tl_row:br_col+br_row]
    n_rows = len(tab_nehodovost)
    n_cols = len(tab_nehodovost[0])


    # stlpce v tabulke frekvencie nehod na useku cesty
    cols_lett = 'GHIJ'

    # iteruj riadky - typ nehody
    for id_row in range(n_rows):
        # zakladna bunka je v prvom stlpci
        base_cell_coord = tab_nehodovost[id_row][0].coordinate

        # stlpec v tabulke usekov s hodnotou frekvencie nehod 
        col_lett_nehoda = cols_lett[id_row]

        # vzorec
        # sum_i vozkm na useku * frekvencia nehod na danom useku / 100 000 000 vozkm
        base_formula = '=SUMPRODUCT(K361:K410,$' + col_lett_nehoda + '$552:$' + col_lett_nehoda + '$601)/100000000'

        for col in range(0, n_cols):
            current_coord = tab_nehodovost[id_row][col].coordinate
            tab_nehodovost[id_row][col].value = opx.formula.translate.Translator(base_formula, \
                                            origin=base_cell_coord).translate_formula(current_coord)


    #### Počet nehôd vo variante V1 - pôvodná cesta

    # Počty nehôd podľa závažnosti v jednotlivých rokoch sú v bunkách `K528:BC531`.

    sheet_name = '09-A Nehodovost - vypocet'
    ws = wb[sheet_name]

    # top left
    tl_col = 'K'
    tl_row = '528'
    # bottom right
    br_col = 'BC'
    br_row = '531'

    tab_nehodovost = ws[tl_col+tl_row:br_col+br_row]
    n_rows = len(tab_nehodovost)
    n_cols = len(tab_nehodovost[0])

    # stlpce v tabulke frekvencie nehod na useku cesty
    cols_lett = 'WXYZ'

    # iteruj riadky - typ nehody
    for id_row in range(n_rows):
        # zakladna bunka je v prvom stlpci
        base_cell_coord = tab_nehodovost[id_row][0].coordinate

        # stlpec v tabulke usekov s hodnotou frekvencie nehod 
        col_lett_nehoda = cols_lett[id_row]

        # vzorec
        # sum_i vozkm na useku * frekvencia nehod na danom useku / 100 000 000 vozkm
        base_formula = '=SUMPRODUCT(K430:K479,$' + col_lett_nehoda + '$552:$' + col_lett_nehoda + '$601)/100000000'

        for col in range(0, n_cols):
            current_coord = tab_nehodovost[id_row][col].coordinate
            tab_nehodovost[id_row][col].value = opx.formula.translate.Translator(base_formula, \
                                            origin=base_cell_coord).translate_formula(current_coord)

    #### Počet nehôd vo variante V1 - nová cesta

    #  Počty nehôd podľa závažnosti v jednotlivých rokoch sú v bunkách `K534:BC537`.

    sheet_name = '09-A Nehodovost - vypocet'
    ws = wb[sheet_name]

    # top left
    tl_col = 'K'
    tl_row = '534'
    # bottom right
    br_col = 'BC'
    br_row = '537'

    tab_nehodovost = ws[tl_col+tl_row:br_col+br_row]
    n_rows = len(tab_nehodovost)
    n_cols = len(tab_nehodovost[0])

    # stlpce v tabulke frekvencie nehod na useku cesty
    cols_lett = 'GHIJ'

    # iteruj riadky - typ nehody
    for id_row in range(n_rows):
        # zakladna bunka je v prvom stlpci
        base_cell_coord = tab_nehodovost[id_row][0].coordinate

        # stlpec v tabulke usekov s hodnotou frekvencie nehod 
        col_lett_nehoda = cols_lett[id_row]

        # vzorec
        # sum_i vozkm na useku * frekvencia nehod na danom useku / 100 000 000 vozkm
        base_formula = '=SUMPRODUCT(K490:K507,$' + col_lett_nehoda + '$612:$' + col_lett_nehoda + '$629)/100000000'

        for col in range(0, n_cols):
            current_coord = tab_nehodovost[id_row][col].coordinate
            tab_nehodovost[id_row][col].value = opx.formula.translate.Translator(base_formula, \
                                            origin=base_cell_coord).translate_formula(current_coord)

    ### Zohľadnenie počtu zranení
    # V0

    sheet_name = '09 Nehodovost'
    ws = wb[sheet_name]

    # top left
    tl_col = 'D'
    tl_row = '21'
    # bottom right
    br_col = 'AK'
    br_row = '23'

    tab_nehodovost = ws[tl_col+tl_row:br_col+br_row]
    n_rows = len(tab_nehodovost)
    n_cols = len(tab_nehodovost[0])

    # zakladna bunka je v prvom riadku a prvom stlpci
    base_cell_coord = tab_nehodovost[0][0].coordinate

    # vzorec
    # povodny vzorec * pocet zranenych
    base_formula = "=D6*INDEX('A1 Parametre'!$D114:$BA114,MATCH(D$19,'A1 Parametre'!$D$113:$BA$113,0))" + \
            "*'A1 Parametre'!$I120"

    # iteruj riadky - typ nehody
    for row in range(n_rows):
        # iteruj stlpce  roky
        for col in range(0, n_cols):
            current_coord = tab_nehodovost[row][col].coordinate
            tab_nehodovost[row][col].value = opx.formula.translate.Translator(base_formula, \
                                            origin=base_cell_coord).translate_formula(current_coord)

    # V1
    sheet_name = '09 Nehodovost'
    ws = wb[sheet_name]

    # top left
    tl_col = 'D'
    tl_row = '30'
    # bottom right
    br_col = 'AK'
    br_row = '32'

    tab_nehodovost = ws[tl_col+tl_row:br_col+br_row]
    n_rows = len(tab_nehodovost)
    n_cols = len(tab_nehodovost[0])

    # zakladna bunka je v prvom riadku a prvom stlpci
    base_cell_coord = tab_nehodovost[0][0].coordinate

    # vzorec
    # povodny vzorec * pocet zranenych
    base_formula = "=D13*INDEX('A1 Parametre'!$D114:$BA114,MATCH(D$19,'A1 Parametre'!$D$113:$BA$113,0))" + \
            "*'A1 Parametre'!$I120"

    # iteruj riadky - typ nehody
    for row in range(n_rows):
        # iteruj stlpce  roky
        for col in range(0, n_cols):
            current_coord = tab_nehodovost[row][col].coordinate
            tab_nehodovost[row][col].value = opx.formula.translate.Translator(base_formula, \
                                            origin=base_cell_coord).translate_formula(current_coord)
            
            
    cba_name = cba_filename.split('.')[0]
    
    # ulozenie vysledku
    wb.save(out_path + cba_name + '_fix_nehodovost.xlsx')